In [ ]:
# 셀레늄 import

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time

# beautiful soup import
from bs4 import BeautifulSoup
import requests


from tqdm import tqdm_notebook



In [ ]:
# 클릭
def select_click(where, driver):
    button = driver.find_element_by_css_selector(str(where))
    actions = webdriver.ActionChains(driver)
    return actions.click(button).perform()


### 클래스101 채용 페이지 크롤링


링크 : https://www.rocketpunch.com/jobs/66401/Product-Analyst%EC%9D%B8%ED%84%B4

링크 구성 : https://www.rocketpunch.com/ + 채용(jobs) + 등록번호(클래스101 자리 - 66401 ? 인건가? + 채용공고 명 (프로덕트 애널리스트 인턴)

In [ ]:


chromedriver = '/Users/taewoongkong/workspace/selenium_learning/chromedriver'
main_driver = webdriver.Chrome(chromedriver)

# n억 이상 투자 받은 스타트업만 보기
funding_amount = "10000000000"  # 100억 10000000000
main_driver.get(f'https://www.rocketpunch.com/companies?funding_volume={funding_amount}-')
time.sleep(4)

# 첫 클릭을 통해 팝업 광고창 뜨게 유도
select_click('#search-results > div.ui.blank.right.floated.segment > div > a:nth-child(4)', main_driver)
time.sleep(3)


# 팝업 광고창 제거
select_click('#company_signup_popup > div:nth-child(1) > i', main_driver)
time.sleep(1)


# 본격적으로 크롤링 시작

company_keyword_list = [] # 복지혜택 크롤링을 위해 검색하기 위한 회사영문명 저장소

while True:
    try:
        # 한 창에 20개씩 나오는 데, 이상하게 html 넘버가 1,3,5 홀수로 39까지 가있음
        for num in range(1,40,2):
            company_link = main_driver.find_element_by_css_selector(f'#company-list > div:nth-child({num}) > div.content > a').get_attribute('href')
            company_keyword_list.append(company_link.split('/')[-1])

        # 다 모았으면 다음 참으로 넘어감
        time.sleep(1)
        select_click('#search-results > div.ui.blank.right.floated.segment > div > a:nth-child(4)', main_driver)
        time.sleep(3)
        
    
    except Exception as ex:
        print(ex)
        print("===========end============")
        break


In [ ]:
for company_name in tqdm_notebook(company_keyword_list):
    
    parsing_url= f"https://www.rocketpunch.com/companies/{company_name}/jobs"
    req = requests.get(parsing_url)
    html = req.content
    soup = BeautifulSoup(html, 'html.parser')
    temp_data = soup.select('#page-benefits > div > div')
    bene_list = []

    

    # 사내 복지 혜택 삽입
    try:
        # 회사 명 정보 들어가야 구분 됨
        bene_list.append('회사명')
        bene_list.append(company_name)
        for benefit in temp_data[0].get_text().split('\n')[1::2]:
            bene_list.append(benefit)
    except:
        print(f'========={company_name}는 채용공고 정보가 없음============')
    # dict 화 작업
    benefit_dict = {bene_list[item]: bene_list[item + 1] for item in range(0, len(bene_list), 2)}
    print(benefit_dict)
    #benefit_df = pd.DataFrame.from_dict(benefit_dict, orient='index')